In [1]:
# Импортируем необходимые библиотеки:
# pandas для манипуляции данными,
# scikit-surprise для функциональности рекомендательных систем, такой как чтение датасетов и применение алгоритмов.
import pandas as pd
from surprise import Dataset, Reader, KNNBasic

# Загружаем датасет с рейтингами фильмов.
ratings = pd.read_csv("ratings.csv")

# Создаем таблицу сводки, чтобы увидеть рейтинги, которые пользователи ставят каждому фильму.
# Индекс: userId, Колонки: movieId, Значения: рейтинг.
user_movie_rating = ratings.pivot_table(index='userId', columns='movieId', values='rating')
# Показываем первые 100 строк для быстрого просмотра.
user_movie_rating.head(100)  

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Определяем объект Reader, указывая шкалу рейтинга.
reader = Reader(rating_scale=(0.5, 5))  # Рейтинги фильмов варьируются от 0.5 до 5.

# Загружаем данные из DataFrame.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Строим полный тренировочный набор данных.
trainset = data.build_full_trainset()

# Создаем экземпляр алгоритма KNNBasic.
algorithm = KNNBasic()
 # Обучаем модель на тренировочном наборе.
algorithm.fit(trainset) 

# Подготавливаем пустой список для тестового набора данных.
testset = []

# Проходим по всем пользователям и фильмам для создания тестового набора.
for uid in trainset.all_users():
    for iid in trainset.all_items():
        # Проверяем отсутствие пары (uid, iid) в тренировочном наборе.
        if not trainset.ur.get(uid) or (iid not in map(lambda x:x[0], trainset.ur[uid])):
            # Добавляем в тестовый набор с None в качестве фактического рейтинга.
            testset.append((trainset.to_raw_uid(uid), trainset.to_raw_iid(iid), None))

# Получаем предсказания для отсутствующих значений.
predictions = algorithm.test(testset)

# Подготавливаем список для хранения предсказанных рейтингов.
pred_ratings = []
for pred in predictions:
    pred_ratings.append([pred.uid, pred.iid, pred.est])

Computing the msd similarity matrix...
Done computing similarity matrix.


In [3]:
# Преобразуем список предсказаний в DataFrame.
pred_df = pd.DataFrame(pred_ratings, columns=['userId', 'movieId', 'rating'])

# Объединяем предсказанные рейтинги с исходными рейтингами.
complete_ratings = pd.concat([ratings[['userId', 'movieId', 'rating']], pred_df])

# Создаем итоговую таблицу сводки с полными данными, включающую как фактические, так и предсказанные рейтинги.
complete_user_movie_rating = complete_ratings.pivot_table(index='userId', columns='movieId', values='rating')

# Сортируем столбцы итоговой сводной таблицы согласно исходной таблице сводки рейтингов.
complete_user_movie_rating = complete_user_movie_rating[user_movie_rating.columns]

# Показываем первые 100 строк итоговой таблицы для быстрого просмотра.
complete_user_movie_rating.head(100)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.000000,3.738519,4.000000,2.517905,3.152936,4.000000,3.294218,2.927424,3.125849,3.616527,...,3.500000,3.000000,4.000000,4.000000,3.500000,4.000000,3.500000,3.500000,3.500000,4.000000
2,3.792351,3.546348,3.291295,2.091471,2.883266,4.063147,3.140670,2.754499,2.527696,3.587689,...,3.500000,3.000000,4.000000,4.000000,3.500000,4.000000,3.500000,3.500000,3.500000,4.000000
3,3.690139,3.098197,2.702006,2.572736,2.913526,3.570667,3.090739,2.616661,2.743723,3.788644,...,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557
4,4.028987,3.304664,3.386649,2.381931,3.042423,3.739104,3.167168,2.842209,3.252007,3.480648,...,3.500000,3.000000,4.000000,4.000000,3.500000,4.000000,3.500000,3.500000,3.500000,4.000000
5,4.000000,3.234912,3.281271,2.287627,2.964203,4.008395,3.073042,2.789332,3.216632,3.476004,...,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,5.000000,3.726108,3.144131,2.409863,3.044727,4.169422,3.135008,2.662256,2.930770,3.653743,...,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,3.501557,4.000000
97,4.101034,3.758149,3.333649,2.566772,3.075093,4.069669,3.169691,2.954473,3.259784,3.499573,...,3.500000,3.000000,4.000000,4.000000,3.500000,4.000000,3.500000,3.500000,3.500000,4.000000
98,4.500000,3.767724,3.388503,2.542364,3.132930,4.001737,3.291559,2.818293,3.351560,3.589705,...,3.500000,3.000000,4.000000,4.000000,3.500000,4.000000,3.500000,3.500000,3.500000,4.000000


In [4]:
# Экспортируем итоговую таблицу в CSV файл.
complete_user_movie_rating.to_csv("predicted_ratings.csv")